#Code implementation of the paper "To Act or React?"
https://arxiv.org/pdf/1906.11932v1.pdf

Connect: ahmedrashidqazi@gmail.com


Credentials of the google account made for this colab.<br>
email: toactorreact@gmail.com <br>
password: dcw22100234

###Libraries

In [58]:
import os
import sys
import pandas as pd
import numpy as np
import sklearn
from google.cloud import bigquery
import re

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
client = bigquery.Client('myproject123-319010')

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Data
##Time Frame - 01/2015 - 10/2018

###DA = Most Active 3 thousand Subreddits based on the average number of posts per month

In [16]:
DA = []
DA = pd.read_csv("/content/gdrive/MyDrive/Most Active 3000 Subreddit Names.csv")
DA

[False nan]


###DB = 36 Quarantined/Banned Subreddits

In [7]:
DB = ['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal'] 

####The code below was used to extract the above 36 subreddits

In [17]:
# DB = []

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.2015_12  WHERE quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# years = ['2018_08', '2019_01']
# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.""" + year + """ WHERE quarantine = true """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       x = x['subreddit']
#       print(x.unique())
#       for k in x.unique():
#         if k not in DB:
#           DB.append(k)
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break


# DB.append("WatchPeopleDie")
# DB.append("Physical_Removal")

 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_07 WHERE quarantine = true 
The query for the month 2018_07
Empty Dataframe



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_08 WHERE quarantine = true 
The query for the month 2018_08
Empty Dataframe



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_09 WHERE quarantine = true 
The query for the month 2018_09
['Braincels' 'TheRedPill' 'Ice_Poseidon' 'CringeAnarchy']



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_10 WHERE quarantine = true 
The query for the month 2018_10
['TheRedPill' 'CringeAnarchy' 'Ice_Poseidon' 'Braincels']



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_11 WHERE quarantine = true 
The query for the month 2018_11
['Braincels' 'TheRedPill' 'Ice_Poseidon' 'CringeAnarchy']



 SELECT subreddit FROM fh-bigquery.reddit_posts.2018_12 WHERE quarantine = true 
The query for the month 2018_12
['Braincels' 'TheRedPill' 'Ice_Poseidon' 'CringeAnarchy']



 SELECT subreddit FROM fh-bigquery.reddi

In [18]:
print("Total number of banned/quarantined subreddits", len(DB))
DB

Total number of banned/quarantined subreddits 36


['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal']

###DH = Hateful Subreddits

####Extracting the text, title, score and number of comments from the subreddit r/AgainstHateSubreddits

In [38]:
# text = pd.DataFrame([], columns = ['selftext', 'title'])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND (subreddit = 'AgainstHateSubreddits') """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2015_12  WHERE subreddit = 'AgainstHateSubreddits' """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.""" + year + """ WHERE subreddit = 'AgainstHateSubreddits' """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break

# text.to_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv", index = False)

 SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2016_01 WHERE subreddit = 'AgainstHateSubreddits' 
The query for the month 2016_01
                                              selftext  ... num_comments
0                                                       ...            1
1                                                       ...            4
2                                                       ...            3
3                                            [removed]  ...            1
4                                                       ...            9
..                                                 ...  ...          ...
203  http://np.reddit.com/r/GenderCritical/comments...  ...           22
204                                                     ...            0
205                                                     ...            1
206                                                     ...           17
207                                    

####Analysing the subreddit to extract names of hateful subreddits

In [39]:
text = pd.read_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv")
text

,selftext,title,score,num_comments
0,NaN,/r/WorldPolitics now linking ONLY to anti-immi...,74.0,30.0
1,NaN,A hate subreddit brigade.,45.0,16.0
2,NaN,Action Taken by mods - Claims that /r/Europe i...,30.0,7.0
3,NaN,"""We need to start a trend on twatter #RemoveKe...",10.0,3.0
4,NaN,Test post please ignore,1.0,1.0
...,...,...,...,...
22973,[removed],What the actual f*ck is this sub,1.0,0.0
22974,[removed],Some thoughts I've got from being on this sub,1.0,0.0
22975,[removed],META: Should direct reddit links be banned on ...,18.0,8.0
22976,[removed],Pro-China Protestors Nazi salute in Vancouver ...,1.0,0.0


In [72]:
#Posts below a score of 100 will be dropped
text = text.sort_values(by = "score", ascending = False)
textNew = text[text['score'] > 100]
textNew['selftext'] = textNew['selftext'].fillna(" ")
textNew

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,selftext,title,score,num_comments
7380,,The ACTUAL FUCKING NAZIS AND WHITE NATIONALIST...,43178.0,2141.0
14743,[deleted],"The_Donald is a violent, extremist group being...",38130.0,2071.0
6231,,HanAssholeSolo wished for people to be doxxed ...,30570.0,1710.0
10093,"For those who may not be aware, /r/The_donald,...",Reddit admins have confirmed they are comforta...,30338.0,2017.0
5426,,/r/The_Donald Has Built A Document With The Ad...,28866.0,2375.0
...,...,...,...,...
5389,,/r/uncensorednews defends man who attacked bla...,101.0,15.0
22603,,DotA2 subreddit defending a professional playe...,101.0,19.0
5510,,"""Jews have an unmatched hatred for white peopl...",101.0,16.0
2217,,"Top nazi from /r/nationalsocialism says ""Non-W...",101.0,24.0


In [78]:
matches = {}
for i in range(0, len(textNew)):
  m = []
  x = textNew.iloc[i]
  print(x['title'])
  y = re.findall(r"/r/([^\s/]+)", x['title'])
  z = re.findall(r"/r/([^\s/]+)", x['selftext'])
  m.append([y, z])
  for i in m:
    if i not in matches.keys():
      matches[i] = 1
    else:
      matches[i] = matches[i] + 1

  print(matches)
  break

The ACTUAL FUCKING NAZIS AND WHITE NATIONALISTS /r/the_donald supported have now killed someone in an ACTUAL FUCKING TERRORIST ATTACK in Virginia. Now they have the nerve to blame Democrats and act like they are the real victims.
{'A': 0}


TypeError: ignored